In [1]:
import numpy as np
import pandas as pd
import glob
import urllib2
import json
import collections
from scipy.stats.mstats import gmean, hmean
import pickle

Load Recon3 genes

In [2]:
recon_genes = sorted(pd.read_table('../recon/genes.tsv')['SYMBOL'].tolist())

Convert PaxDB Ensembl protein ID's to gene symbols

In [3]:
# load PaxDB ensembl protein ID's
ensemblprotein = []
for fn in glob.glob('data/*'):
    df = pd.read_table(fn, skiprows=range(11))
    ensemblprotein.extend([x.split('.')[1] for x in df['string_external_id'].tolist()])
ensemblprotein = list(set(ensemblprotein))

In [4]:
# convert to gene symbols
stepsize = 400
convert_genesymbol_ensemblprotein = {}
for i in range(len(ensemblprotein)/stepsize+1):
    if i < len(ensemblprotein)/stepsize:
        string = ','.join(ensemblprotein[(stepsize*i):(stepsize*(i+1))])
    else:
        string = ','.join(ensemblprotein[(stepsize*i):])
    url = 'https://biodbnet-abcc.ncifcrf.gov/webServices/rest.php/biodbnetRestApi.json?method=db2db&input=ensemblproteinid&inputValues=%s&outputs=genesymbol&taxonId=9606&format=row' % string
    data = json.loads(urllib2.urlopen(url).read())
    for j in range(len(data)):
        if str(data[j]['Gene Symbol']) != '-':
            for gene in str(data[j]['Gene Symbol']).split('//'):
                if gene in recon_genes:
                    if gene not in convert_genesymbol_ensemblprotein:
                        convert_genesymbol_ensemblprotein[gene] = [str(data[j]['InputValue'])]
                    elif str(data[j]['InputValue']) not in convert_genesymbol_ensemblprotein[gene]:
                        convert_genesymbol_ensemblprotein[gene].append(str(data[j]['InputValue']))

In [5]:
# subset of recon genes that are in PaxDB data
recon_genes_paxdb = [x for x in recon_genes if x in convert_genesymbol_ensemblprotein]

Get protein abundances for Recon3 genes

In [6]:
# initialize values
genes = []
values = []

# iterate over datasets
for fn in glob.glob('data/*'):
    
    # load data
    df = pd.read_table(fn, skiprows=range(11), index_col = 1)
    df.index = [x.split('.')[1] for x in df.index.tolist()]
    
    # iterate over recon genes
    for gene in recon_genes_paxdb:
        keep = []
        for ensembl in convert_genesymbol_ensemblprotein[gene]:
            if ensembl in df.index:
                if df.loc[ensembl]['abundance'] > 0:
                    keep.append(df.loc[ensembl]['abundance'])
        if len(keep) >= 1:
            if gene not in genes:
                genes.append(gene)
                values.append([np.sum(keep)])
            else:
                values[genes.index(gene)].append(np.sum(keep))

Average protein abundance for Recon3 genes

In [7]:
recongenes_averageabundance = {}
for i,gene in enumerate(genes):
    recongenes_averageabundance[gene] = hmean(values[i])
recongenes_averageabundance['_ALL_'] = hmean([item for sublist in values for item in sublist])

Save data

In [8]:
with open('paxdb.pkl','w') as f:
    pickle.dump(recongenes_averageabundance, f)